# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ki" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
# os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
# !echo $gdrive_path

In [4]:
# Install opus-tools
#! pip install opustools-pkg

Uncomment cell below if notebook is being run for the first time and you need to download the data

In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ki.xml.gz not found. The following files are available for downloading:

 964 KB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ki.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  10 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ki.zip

 274 MB Total size
./JW300_latest_xml_en-ki.xml.gz ... 100% of 964 KB
./JW300_latest_xml_en.zip ... 100% of 263 MBn.zip ... 96% of 263 MB
./JW300_latest_xml_ki.zip ... 100% of 10 MB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-02-19 05:55:36--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2020-02-19 05:55:36 (6.56 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-02-19 05:55:36--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ki.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200

In [5]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [6]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 4277/106699 lines since contained in test set.


,source_sentence,target_sentence
0,© 2015 Watch Tower Bible and Tract Society of ...,© 2015 Watch Tower Bible and Tract Society of ...
1,Two Questions Worth Asking,Ciũria Igĩrĩ cia Bata Tũngĩĩyũria
2,An Answer Worth Considering,Macokio ma Bata


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [7]:
import numpy as np
# drop duplicate translations
df_pp = df.drop_duplicates()

#drop empty lines (alp)
df_pp['source_sentence'].replace('', np.nan, inplace=True)
df_pp['target_sentence'].replace('', np.nan, inplace=True)
df_pp.dropna(subset=['source_sentence'], inplace=True)
df_pp.dropna(subset=['target_sentence'], inplace=True)

# drop conflicting translations
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_

In [8]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.bpe.en <==
These are often d@@ ue to the bo@@ d@@ y’s dir@@ ecting blood to the cor@@ e to support v@@ ital org@@ ans .
The power to per@@ form fe@@ at@@ s like that could come only from God’s enem@@ ies in the spirit real@@ m . ​ — Ex@@ od@@ us 7 : 8 - 12 .
In fact , that is what he promised to do for anointed ones in the new covenant : “ I will put my law with@@ in them , and in their heart I sha@@ ll wr@@ ite it . And I will become their God , and they themselves will become my people . ”
Instead , Sa@@ ul ref@@ used to be mo@@ ld@@ ed .
In@@ clud@@ ed among these were sever@@ al of Jesus ’ family members , who at first had not taken ser@@ i@@ ously the pos@@ sib@@ ility that one of their relati@@ ves could be the Mess@@ iah .
B@@ enj@@ am@@ in , ag@@ ed 33 , is An@@ n@@ e - R@@ ak@@ el@@ ’s brother .
Th@@ erefore , let us consider how we can over@@ come ob@@ st@@ ac@@ les that make it challeng@@ ing to listen to God’s vo@@ ice .
You will have increas@@ ed satis@@ fac@@ ti



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [11]:
# Install JoeyNMT
#! git clone https://github.com/joeynmt/joeynmt.git
#! cd joeynmt; pip3 install .

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [9]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("../../joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

In [10]:
# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

In [11]:
! sudo chmod 777 ../../joeynmt/data/

In [12]:
# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

bpe.codes.4000	dev.en	     test.bpe.ki     test.ki	   train.en
dev.bpe.en	dev.ki	     test.en	     train.bpe.en  train.ki
dev.bpe.ki	test.bpe.en  test.en-any.en  train.bpe.ki  vocab.txt


In [6]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

In [13]:
# Create that vocab using build_vocab
! sudo chmod 777 ../../joeynmt/scripts/build_vocab.py
! ../../joeynmt/scripts/build_vocab.py ../../joeynmt/data/$src$tgt/train.bpe.$src ../../joeynmt/data/$src$tgt/train.bpe.$tgt --output_path ../../joeynmt/data/$src$tgt/vocab.txt

In [14]:
# Some output
! echo "BPE Kikuyu Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 ../../joeynmt/data/$src$tgt/vocab.txt  # Herman

BPE Kikuyu Sentences
N@@ go ĩrĩa nene ya wĩtĩkio ( Rora kĩbungo gĩa 12 - 14 )
N@@ gũ@@ b@@ ia ya ũh@@ onokio ( Rora kĩbungo gĩa 15 - 18 )
Nĩ ny@@ on@@ ete atĩ andũ nĩ math@@ ik@@ agĩrĩria wega mangĩ@@ ona atĩ nĩ wendete Bibilia mũno na nĩ ũr@@ er@@ utanĩria kũm@@ ateith@@ ia . ”
R@@ ũ@@ hi@@ ũ rwa roho ( Rora kĩbungo gĩa 19 - 20 )
Kũg@@ erera ũteithio wa Jehova , no tũhote kwĩ@@ hand@@ a wega tw@@ ĩt@@ irie Shaitani , thũ itũ .
Combined BPE Vocab
Hih@@
ʺ
KARAT@@
ē@@
Ó@@
greg@@
artic@@
✔
ā@@
iraeli


In [9]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
#! cp train.* "$gdrive_path"
#! cp test.* "$gdrive_path"
#! cp dev.* "$gdrive_path"
#! cp bpe.codes.4000 "$gdrive_path"
#! ls "$gdrive_path"

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [15]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "models/{name}_transformer/1000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path="n/a", source_language=source_language, target_language=target_language)
with open("../../joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [16]:
! ls -la ../../joeynmt

total 136
drwxr-xr-x 11 root root  4096 Oct 25 12:22 .
drwxrwxrwx 11 root root  4096 Feb 19 06:15 ..
drwxr-xr-x  8 root root  4096 Oct 24 15:14 .git
-rw-r--r--  1 root root    49 Oct 24 15:14 .gitattributes
drwxr-xr-x  3 root root  4096 Oct 24 15:14 .github
-rw-r--r--  1 root root    71 Oct 24 15:14 .gitignore
-rw-r--r--  1 root root 13514 Oct 24 15:14 .pylintrc
-rw-r--r--  1 root root   159 Oct 24 15:14 .readthedocs.yml
-rw-r--r--  1 root root   542 Oct 24 15:14 .travis.yml
-rwxrw-rwx  1 root root  3354 Oct 24 15:14 CODE_OF_CONDUCT.md
-rwxrw-rwx  1 root root  1071 Oct 24 15:14 LICENSE
-rwxrw-rwx  1 root root 13286 Oct 24 15:14 README.md
-rwxrw-rwx  1 root root  8229 Oct 24 15:14 benchmarks.md
drwxrw-rwx  3 root root  4096 Feb 19 06:30 configs
drwxrwxrwx  8 root root  4096 Feb 19 06:35 data
drwxrw-rwx  4 root root  4096 Oct 24 15:14 docs
-rwxrw-rwx  1 root root 14373 Oct 24 15:14 joey-small.png
drwxrw-rwx  3 root root  4096 Oct 24 16:35 joeynmt
drwxrwxrwx  8 root root  4096 Feb 19 07:5

In [17]:
! sudo chmod 777 ../../joeynmt/models

In [18]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
! cd ../../joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/espoir_mur_gmail_com/.

2020-02-20 07:37:58,484 cfg.name                           : enki_transformer
2020-02-20 07:37:58,484 cfg.data.src                       : en
2020-02-20 07:37:58,484 cfg.data.trg                       : ki
2020-02-20 07:37:58,484 cfg.data.train                     : data/enki/train.bpe
2020-02-20 07:37:58,484 cfg.data.dev                       : data/enki/dev.bpe
2020-02-20 07:37:58,484 cfg.data.test                      : data/enki/test.bpe
2020-02-20 07:37:58,484 cfg.data.level                     : bpe
2020-02-20 07:37:58,484 cfg.data.lowercase                 : False
2020-02-20 07:37:58,484 cfg.data.max_sent_length           : 100
2020-02-20 07:37:58,484 cfg.data.src_vocab                 : data/enki/vocab.txt
2020-02-20 07:37:58,485 cfg.data.trg_vocab                 : data/enki/vocab.txt
2020-02-20 07:37:58,485 cfg.testing.beam_size              : 5
2020-02-20 07:37:58,485 cfg.testing.alpha                  : 1.0
2020-02-20 07:37:58,485 cfg.training.random_seed           : 42
202

2020-02-20 07:44:44,480 Epoch   1 Step:     1100 Batch Loss:     3.988148 Tokens per Sec:     7646, Lr: 0.000300
2020-02-20 07:44:44,751 Epoch   1: total training loss 5155.24
2020-02-20 07:44:44,752 EPOCH 2
2020-02-20 07:45:13,301 Epoch   2 Step:     1200 Batch Loss:     3.747825 Tokens per Sec:     7729, Lr: 0.000300
2020-02-20 07:45:42,119 Epoch   2 Step:     1300 Batch Loss:     3.764219 Tokens per Sec:     7797, Lr: 0.000300
2020-02-20 07:46:10,766 Epoch   2 Step:     1400 Batch Loss:     3.736711 Tokens per Sec:     7631, Lr: 0.000300
2020-02-20 07:46:39,691 Epoch   2 Step:     1500 Batch Loss:     3.946311 Tokens per Sec:     7654, Lr: 0.000300
2020-02-20 07:47:08,246 Epoch   2 Step:     1600 Batch Loss:     3.595036 Tokens per Sec:     7729, Lr: 0.000300
2020-02-20 07:47:36,814 Epoch   2 Step:     1700 Batch Loss:     3.649236 Tokens per Sec:     7372, Lr: 0.000300
2020-02-20 07:48:05,665 Epoch   2 Step:     1800 Batch Loss:     3.596375 Tokens per Sec:     7804, Lr: 0.000300
2

2020-02-20 08:03:40,183 Epoch   4 Step:     4100 Batch Loss:     3.001262 Tokens per Sec:     7681, Lr: 0.000300
2020-02-20 08:04:09,053 Epoch   4 Step:     4200 Batch Loss:     2.729484 Tokens per Sec:     7615, Lr: 0.000300
2020-02-20 08:04:37,737 Epoch   4 Step:     4300 Batch Loss:     2.872530 Tokens per Sec:     7649, Lr: 0.000300
2020-02-20 08:05:03,603 Epoch   4: total training loss 3117.84
2020-02-20 08:05:03,603 EPOCH 5
2020-02-20 08:05:06,619 Epoch   5 Step:     4400 Batch Loss:     2.815613 Tokens per Sec:     7475, Lr: 0.000300
2020-02-20 08:05:35,235 Epoch   5 Step:     4500 Batch Loss:     2.658053 Tokens per Sec:     7700, Lr: 0.000300
2020-02-20 08:06:03,907 Epoch   5 Step:     4600 Batch Loss:     2.678884 Tokens per Sec:     7692, Lr: 0.000300
2020-02-20 08:06:32,440 Epoch   5 Step:     4700 Batch Loss:     2.239846 Tokens per Sec:     7679, Lr: 0.000300
2020-02-20 08:07:01,316 Epoch   5 Step:     4800 Batch Loss:     2.742702 Tokens per Sec:     7599, Lr: 0.000300
2

2020-02-20 08:22:05,804 	Hypothesis: Ariũ a Ithe witũ arĩa maikaraga Amerika ( U.S ) nĩ maahoririe andũ milioni nyingĩ mũno , na nĩ maakenire mũno nĩ ũndũ wa gũkorũo na andũ milioni nyingĩ mũno , na nĩ maanyitirũo ũgeni nĩ andũ arĩa maikaraga makuĩte ngoro mũno .
2020-02-20 08:22:05,804 Example #1
2020-02-20 08:22:05,805 	Source:     Yes .
2020-02-20 08:22:05,805 	Reference:  Ĩĩ .
2020-02-20 08:22:05,805 	Hypothesis: Ĩĩ .
2020-02-20 08:22:05,805 Example #2
2020-02-20 08:22:05,805 	Source:     He enjoyed special intimacy with Jehovah , who enabled him to display “ awesome power ” as he led the Israelites to the Promised Land . ​ — Deut .
2020-02-20 08:22:05,805 	Reference:  Aarĩ na ũkuruhanu wa hakuhĩ na Jehova , na nĩ aamũheire ũhoti wa gwĩka “ maũndũ manene ma kũguoyohania ” rĩrĩa aatongoragia Aisiraeli mathiĩ Bũrũri wa Kĩĩranĩro . — Gũcok .
2020-02-20 08:22:05,805 	Hypothesis: Nĩ aakenire mũno nĩ ũndũ wa gũkorũo na wendo wa mwanya , Jehova nĩ aamwĩrire atĩ nĩ ‘ aakenagĩra ’ na akĩmũh

...

2020-02-20 09:44:34,643 Epoch  19 Step:    20100 Batch Loss:     1.911918 Tokens per Sec:     7558, Lr: 0.000300
2020-02-20 09:45:03,459 Epoch  19 Step:    20200 Batch Loss:     2.009794 Tokens per Sec:     7638, Lr: 0.000300
2020-02-20 09:45:32,382 Epoch  19 Step:    20300 Batch Loss:     2.239769 Tokens per Sec:     7841, Lr: 0.000300
2020-02-20 09:46:01,129 Epoch  19 Step:    20400 Batch Loss:     1.981424 Tokens per Sec:     7631, Lr: 0.000300
2020-02-20 09:46:30,005 Epoch  19 Step:    20500 Batch Loss:     1.979983 Tokens per Sec:     7786, Lr: 0.000300
2020-02-20 09:46:58,752 Epoch  19 Step:    20600 Batch Loss:     1.382899 Tokens per Sec:     7669, Lr: 0.000300
2020-02-20 09:47:27,195 Epoch  19 Step:    20700 Batch Loss:     1.610619 Tokens per Sec:     7357, Lr: 0.000300
2020-02-20 09:47:56,029 Epoch  19 Step:    20800 Batch Loss:     1.765529 Tokens per Sec:     7841, Lr: 0.000300
2020-02-20 09:48:12,791 Epoch  19: total training loss 2010.95
2020-02-20 09:48:12,792 EPOCH 20


2020-02-20 10:03:17,380 Epoch  21: total training loss 1963.73
2020-02-20 10:03:17,380 EPOCH 22
2020-02-20 10:03:29,098 Epoch  22 Step:    23100 Batch Loss:     1.923240 Tokens per Sec:     7282, Lr: 0.000300
2020-02-20 10:03:58,047 Epoch  22 Step:    23200 Batch Loss:     1.682064 Tokens per Sec:     7787, Lr: 0.000300
2020-02-20 10:04:26,859 Epoch  22 Step:    23300 Batch Loss:     1.587517 Tokens per Sec:     7528, Lr: 0.000300
2020-02-20 10:04:55,573 Epoch  22 Step:    23400 Batch Loss:     1.631523 Tokens per Sec:     7699, Lr: 0.000300
2020-02-20 10:05:24,618 Epoch  22 Step:    23500 Batch Loss:     1.568314 Tokens per Sec:     7826, Lr: 0.000300
2020-02-20 10:05:53,717 Epoch  22 Step:    23600 Batch Loss:     1.872469 Tokens per Sec:     7666, Lr: 0.000300
2020-02-20 10:06:22,576 Epoch  22 Step:    23700 Batch Loss:     1.770639 Tokens per Sec:     7607, Lr: 0.000300
2020-02-20 10:06:51,321 Epoch  22 Step:    23800 Batch Loss:     1.423158 Tokens per Sec:     7790, Lr: 0.000300


2020-02-20 10:22:25,577 Epoch  24 Step:    26100 Batch Loss:     1.864329 Tokens per Sec:     7598, Lr: 0.000300
2020-02-20 10:22:54,292 Epoch  24 Step:    26200 Batch Loss:     2.075580 Tokens per Sec:     7762, Lr: 0.000300
2020-02-20 10:23:23,104 Epoch  24 Step:    26300 Batch Loss:     1.820727 Tokens per Sec:     7566, Lr: 0.000300
2020-02-20 10:23:34,862 Epoch  24: total training loss 1887.59
2020-02-20 10:23:34,863 EPOCH 25
2020-02-20 10:23:52,007 Epoch  25 Step:    26400 Batch Loss:     1.723013 Tokens per Sec:     7676, Lr: 0.000300
2020-02-20 10:24:20,607 Epoch  25 Step:    26500 Batch Loss:     1.900988 Tokens per Sec:     7592, Lr: 0.000300
2020-02-20 10:24:49,418 Epoch  25 Step:    26600 Batch Loss:     1.875849 Tokens per Sec:     7739, Lr: 0.000300
2020-02-20 10:25:18,264 Epoch  25 Step:    26700 Batch Loss:     1.768392 Tokens per Sec:     7637, Lr: 0.000300
2020-02-20 10:25:46,990 Epoch  25 Step:    26800 Batch Loss:     1.530995 Tokens per Sec:     7613, Lr: 0.000300


2020-02-20 10:41:19,132 Epoch  27 Step:    29100 Batch Loss:     1.771853 Tokens per Sec:     7520, Lr: 0.000300
2020-02-20 10:41:47,661 Epoch  27 Step:    29200 Batch Loss:     1.573860 Tokens per Sec:     7592, Lr: 0.000300
2020-02-20 10:42:16,152 Epoch  27 Step:    29300 Batch Loss:     1.881424 Tokens per Sec:     7574, Lr: 0.000300
2020-02-20 10:42:44,940 Epoch  27 Step:    29400 Batch Loss:     1.846929 Tokens per Sec:     7745, Lr: 0.000300
2020-02-20 10:43:13,726 Epoch  27 Step:    29500 Batch Loss:     1.933888 Tokens per Sec:     7783, Lr: 0.000300
2020-02-20 10:43:42,499 Epoch  27 Step:    29600 Batch Loss:     1.905533 Tokens per Sec:     7771, Lr: 0.000300
2020-02-20 10:43:52,443 Epoch  27: total training loss 1841.99
2020-02-20 10:43:52,443 EPOCH 28
2020-02-20 10:44:11,122 Epoch  28 Step:    29700 Batch Loss:     1.790683 Tokens per Sec:     7548, Lr: 0.000300
2020-02-20 10:44:40,124 Epoch  28 Step:    29800 Batch Loss:     1.714535 Tokens per Sec:     7697, Lr: 0.000300


2020-02-20 11:00:12,807 Epoch  30 Step:    32100 Batch Loss:     1.610383 Tokens per Sec:     7718, Lr: 0.000300
2020-02-20 11:00:41,777 Epoch  30 Step:    32200 Batch Loss:     1.876286 Tokens per Sec:     7646, Lr: 0.000300
2020-02-20 11:01:10,048 Epoch  30 Step:    32300 Batch Loss:     1.924929 Tokens per Sec:     7672, Lr: 0.000300
2020-02-20 11:01:38,556 Epoch  30 Step:    32400 Batch Loss:     1.818684 Tokens per Sec:     7576, Lr: 0.000300
2020-02-20 11:02:07,424 Epoch  30 Step:    32500 Batch Loss:     1.408155 Tokens per Sec:     7746, Lr: 0.000300
2020-02-20 11:02:36,124 Epoch  30 Step:    32600 Batch Loss:     1.853153 Tokens per Sec:     7527, Lr: 0.000300
2020-02-20 11:03:04,892 Epoch  30 Step:    32700 Batch Loss:     1.727104 Tokens per Sec:     7646, Lr: 0.000300
2020-02-20 11:03:33,842 Epoch  30 Step:    32800 Batch Loss:     2.103340 Tokens per Sec:     7715, Lr: 0.000300
2020-02-20 11:04:02,603 Epoch  30 Step:    32900 Batch Loss:     1.930896 Tokens per Sec:     76

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [20]:
# Output our validation accuracy
! cat "../../joeynmt/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 99303.99219	PPL: 50.35875	bleu: 1.98324	LR: 0.00030000	*
Steps: 2000	Loss: 83300.25781	PPL: 26.77751	bleu: 3.17995	LR: 0.00030000	*
Steps: 3000	Loss: 74742.76562	PPL: 19.10268	bleu: 5.37697	LR: 0.00030000	*
Steps: 4000	Loss: 69263.21094	PPL: 15.38773	bleu: 8.07771	LR: 0.00030000	*
Steps: 5000	Loss: 65603.03906	PPL: 13.31800	bleu: 9.66573	LR: 0.00030000	*
Steps: 6000	Loss: 62489.73438	PPL: 11.77814	bleu: 11.01826	LR: 0.00030000	*
Steps: 7000	Loss: 60542.17578	PPL: 10.90676	bleu: 12.02475	LR: 0.00030000	*
Steps: 8000	Loss: 58300.29297	PPL: 9.98320	bleu: 13.29180	LR: 0.00030000	*
Steps: 9000	Loss: 56454.33594	PPL: 9.28175	bleu: 13.84273	LR: 0.00030000	*
Steps: 10000	Loss: 54946.10156	PPL: 8.74538	bleu: 14.93520	LR: 0.00030000	*
Steps: 11000	Loss: 53487.99219	PPL: 8.25632	bleu: 15.73370	LR: 0.00030000	*
Steps: 12000	Loss: 52413.11719	PPL: 7.91340	bleu: 16.22676	LR: 0.00030000	*
Steps: 13000	Loss: 51187.14453	PPL: 7.53963	bleu: 17.23973	LR: 0.00030000	*
Steps: 14000	Loss: 

In [ ]:
# Test our model
! cd ../../joeynmt; python3 -m joeynmt test "models/${src}${tgt}_transformer/config.yaml"

/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/espoir_mur_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/espoir_mur_gmail_com/.